In [ ]:
import numpy as np

def LCM_infer(X, opts=None):
    """
    Particle filtering or local MAP inference for the latent cause model of associative learning.
    
    Parameters:
    X (numpy.ndarray): [T x D] stimulus inputs, where T is the number of timepoints and D is the number of stimulus features.
                       Binary features are assumed. The first feature (column 0) is the US, and the rest are CSs.
    opts (dict): Optional dictionary containing various options (see LCM_opts). If opts['M'] == 1, local MAP is used.
    
    Returns:
    dict: A dictionary with results, containing:
        - 'opts': Options used
        - 'V': [T x 1] US predictions
        - 'post': [T x K] latent cause posterior probabilities
    """
    # Set parameters
    opts = LCM_opts(opts) if opts else {}
    M = opts.get('M', 1)
    a = opts.get('a', 1)
    b = opts.get('b', 1)
    alpha = opts.get('alpha', 0)
    stickiness = opts.get('stickiness', 0)
    K = opts.get('K', 1 if alpha == 0 else opts.get('K'))

    results = {'opts': opts}
    T, D = X.shape
#     print(T,D)

    # Initialization
    post = np.zeros((1, K))
    post[0, 0] = 1
    post0 = np.zeros((M, K))
    post0[:, 0] = 1
    N = np.zeros((M, K, D))  # feature-cause co-occurrence counts
    B = np.zeros((M, K, D))  # co-occurrence counts for features absent
    Nk = np.zeros((M, K))     # cause counts
    results['post'] = np.hstack([np.ones((T, 1)), np.zeros((T, K - 1))])
    results['V'] = np.zeros((T, 1))
    z = np.ones(M, dtype=int)

    # Loop over trials
    for t in range(T):
        # Calculate likelihood
        lik = N.copy()
        lik[:, :, X[t, :] == 0] = B[:, :, X[t, :] == 0]
        lik = (lik + a) / (Nk[:, :, np.newaxis] + a + b)
#         print(lik)

        if alpha > 0:  # Only update if concentration parameter is non-zero
            # Calculate CRP prior
            prior = Nk.copy()
            for m in range(M):
                prior[m, z[m] - 1] += stickiness  # Add stickiness
                prior[m, np.where(prior[m, :] == 0)[0][0]] = alpha  # New latent cause
            prior /= np.sum(prior, axis=1, keepdims=True)

            # Posterior conditional on CS only
            post = prior * np.prod(lik[:, :, 1:D], axis=2)
            post0 = post / np.sum(post, axis=1, keepdims=True)

            # Posterior conditional on CS and US
            post *= lik[:, :, 0]
            post /= np.sum(post)

        results['post'][t, :] = np.mean(post / np.sum(post, axis=1, keepdims=True), axis=0)

        # Posterior predictive mean for US
        pUS = (N[:, :, 0] + a) / (Nk + a + b)
        results['V'][t, 0] = np.dot(post0.flatten(), pUS.flatten()) / M

        # Sample new particles
        x1 = X[t, :] == 1 # indices where obs occurred in a trial
        x0 = X[t, :] == 0 # indices where obs didn't occur in a trial

        if M == 1:
            z = np.argmax(post) + 1  # Max a posteriori
            Nk[0, z - 1] += 1
            N[0, z - 1, x1] += 1
            B[0, z - 1, x0] += 1
        else:
            Nk_old, N_old, B_old = Nk.copy(), N.copy(), B.copy()
            for m in range(M):
                row = np.min(np.where(np.random.rand() < np.cumsum(np.sum(post, axis=1)))[0])
                # out of the N hypothesis, 

                Nk[m, :] = Nk_old[row, :]
                N[m, :, :] = N_old[row, :, :]
                B[m, :, :] = B_old[row, :, :]
                a = np.random.rand()
                col = np.min(np.where(a < np.cumsum(post[row, :] / np.sum(post[row, :])))[0])
                # why do np.cumsum()? because we want to random select within expanded latent causes
                
                Nk[m, col] += 1
                N[m, col, x1] += 1
                B[m, col, x0] += 1
                
                # why assign observations to randomly selected latent cause? 
                # because we believe that even if we do this, results will converge

    # Remove unused particles
    unused = np.mean(results['post'], axis=0) == 0
    results['post'] = results['post'][:, ~unused]

    return results

def LCM_opts(opts):
    """Default options for the latent cause model."""
    default_opts = {
        'M': 1,          # Number of particles
        'a': 1.0,        # Parameter for likelihood
        'b': 1.0,        # Parameter for likelihood
        'alpha': 0.0,    # Concentration parameter for CRP
        'stickiness': 0.0,  # Stickiness for CRP prior
        'K': 5           # Initial number of latent causes
    }
    if opts is None:
        return default_opts
    default_opts.update(opts)
    return default_opts


In [ ]:
X = np.array([[1, 0, 1], [0, 1, 0], [1, 1, 0]])  # Example stimulus input
opts = {'M': 11, 'alpha': 0.5}  # Example options
results = LCM_infer(X, opts)
print(results)


In [ ]:
np.random.rand()

In [ ]:
results['post']

In [ ]:
results['V']

In [ ]:
X[0,:]==1